In [1]:
import numpy as np
import pandas as pd
import psycopg2
import os

In [2]:
cwd = os.getcwd()
path = os.path.join(cwd,r'HRSS_normal_standard.csv')

#HRSSnormalstandard.csv contains normal runs without failures and not optimized. Runs with failures and/or optimized could also be selected
df = pd.read_csv(path)
df

,Timestamp,Labels,I_w_BLO_Weg,O_w_BLO_power,O_w_BLO_voltage,I_w_BHL_Weg,O_w_BHL_power,O_w_BHL_voltage,I_w_BHR_Weg,O_w_BHR_power,O_w_BHR_voltage,I_w_BRU_Weg,O_w_BRU_power,O_w_BRU_voltage,I_w_HR_Weg,O_w_HR_power,O_w_HR_voltage,I_w_HL_Weg,O_w_HL_power,O_w_HL_voltage
0,0.000000,0,-107,0,0,0,0,0,-1268,0,0,-26,84,11,0,7168,26,0,7720,24
1,0.045998,0,-107,0,0,0,0,0,-1268,0,0,-26,84,11,0,7168,26,0,7720,24
2,0.151001,0,-107,0,0,0,0,0,-1268,0,0,-26,84,11,0,7168,26,0,7720,24
3,0.206001,0,-107,0,0,0,0,0,-1268,0,0,29,21725,54,0,6726,26,0,10756,26
4,0.264000,0,-107,0,0,0,0,0,-1268,0,0,29,21725,54,0,6726,26,0,10756,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25902,14.847000,0,8,0,0,-547,0,0,-873,0,0,-496,23655,57,0,5322,24,0,5440,24
25903,14.904999,0,8,0,0,-547,0,0,-873,0,0,-496,23655,57,0,5322,24,0,5440,24
25904,14.960999,0,8,0,0,-547,0,0,-873,0,0,-575,21670,55,0,5504,24,0,5440,24
25905,15.005997,0,8,0,0,-547,0,0,-873,0,0,-575,21670,55,0,5504,24,0,5440,24


In [3]:
import psycopg2
from io import StringIO

# Connection parameters
param_dic = {
    "host"      : "localhost",
    "database"  : "sensordata",
    "user"      : "postgres",
    "password"  : "JNt8hSW4l+MJ]p;.g[WG"
}

#Credit to Naysan Saran. https://naysan.ca/2020/06/21/pandas-to-postgresql-using-psycopg2-copy_from/

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn


def copy_from_stringio(conn, df, table):
    """
    Here we are going save the dataframe in memory 
    and use copy_from() to copy it to the table
    """
    # save dataframe to an in memory buffer
    buffer = StringIO()
    df.to_csv(buffer, index_label='id', header=False)
    buffer.seek(0)

    cursor = conn.cursor()
    try:
        cursor.copy_from(buffer, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:

        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_stringio() done")
    cursor.close()

In [5]:
conn = connect(param_dic) # connect to the database
cursor = conn.cursor() # set cursor

# CREATE TABLE
list1 = list(df.keys()) #use column labels for SQL query
string1 = "CREATE TABLE high_storage_system (id int"
for label in list1:
    string1 += (", " + label + " decimal")
string1 = string1 + ");"
print(string1)
cursor.execute(string1) #execute SQL query

copy_from_stringio(conn, df, "high_storage_system") # copy the dataframe to SQL
conn.close() # close the connection

Connecting to the PostgreSQL database...
Connection successful
CREATE TABLE high_storage_system (id int, Timestamp decimal, Labels decimal, I_w_BLO_Weg decimal, O_w_BLO_power decimal, O_w_BLO_voltage decimal, I_w_BHL_Weg decimal, O_w_BHL_power decimal, O_w_BHL_voltage decimal, I_w_BHR_Weg decimal, O_w_BHR_power decimal, O_w_BHR_voltage decimal, I_w_BRU_Weg decimal, O_w_BRU_power decimal, O_w_BRU_voltage decimal, I_w_HR_Weg decimal, O_w_HR_power decimal, O_w_HR_voltage decimal, I_w_HL_Weg decimal, O_w_HL_power decimal, O_w_HL_voltage decimal);
copy_from_stringio() done
